# Descriptive Statistics & Balance Tables

`DTable()` allows to display descriptive statistics for a set of variables in the same layout. `DTable()` inherits from the `MTable` base class, which provides all the core output functionality. This means that `ETable` can generate tables in multiple formats (HTML/GT, docx, LaTeX). `BTable()` inherits from `DTable()` to display simple Balance Tables adding statistical tests for treatment comparisons. 

## Basic Usage of `DTable()`
Specify the variables you want to display the descriptive statistics for. Here we also directly define variable labels and set these as default labels (see [Setting defaults](ETable.ipynb#sec-setting-defaults) in the ETable documentation).

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import maketables as mt

# Load sample dataset
df = pd.read_csv("../data/salaries.csv")

# Define variable labels
labels = {
    "logwage": "ln(Wage)",
    "wage": "Wage",
    "age": "Age",
    "female": "Female",
    "tenure": "Years of Tenure",
    "occupation": "Occupation",
    "worker_type": "Worker Type",
    "education": "Education Level"
}

# Set default labels 
mt.MTable.DEFAULT_LABELS = labels


In [2]:
mt.DTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    caption="Descriptive statistics",
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8423770>

Choose the set of statistics to be displayed with `stats`. You can use any pandas aggregation functions.


In [3]:
mt.DTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    stats=["count", "mean", "std", "min", "max"],
    caption="Descriptive statistics",
    
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8422900>

## Summarize by characteristics in columns and rows
You can summarize by characteristics using the `bycol` argument when groups are to be displayed in columns. When the number of observations is the same for all variables in a group, you can also opt to display the number of observations only once for each group byin a separate line at the bottom of the table with `counts_row_below==True`.


In [4]:
# Generate a categorical variable for gender from the dummy variable
df["gender"] = df["female"].map({0: "Male", 1: "Female"})

mt.DTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    bycol=["worker_type","gender"],
    stats=["count", "mean", "std"],
    caption="Descriptive statistics by worker type and gender",
    stats_labels={"count": "Number of observations"},
    counts_row_below=True,
    digits=2)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8422900>

You can also use custom aggregation functions to compute further statistics or affect how statistics are presented. Pyfixest provides two such functions `mean_std` and `mean_newline_std` which compute the mean and standard deviation and display both the same cell (either with line break between them or not). This allows to have more compact tables when you want to show statistics for many characteristcs in the columns.

You can also hide the display of the statistics labels in the header with `hide_stats_labels=True`. In that case a table note will be added naming the statistics displayed using its label (if you have not provided a custom note).


In [5]:
mt.DTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    bycol=["worker_type", "gender"],
    stats=["mean_newline_std", "count"],
    caption="Descriptive statistics by worker type and gender",
    stats_labels={"count": "Number of observations"},
    counts_row_below=True,
    hide_stats=True,
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8423770>

You can also split by characteristics in both columns and rows. Note that you can only use one grouping variable in rows, but several in columns (as shown above).


In [6]:
mt.DTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    bycol=["worker_type"],
    byrow="gender",
    stats=["count", "mean", "std"],
    caption="Descriptive statistics by worker type and gender",
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8423770>

## Number formatting

DTable supports flexible number formatting via the `format_spec` argument. You can control formatting at three levels passing a dictionary:

- Key types accepted:
  - `('var', 'stat')` — per-variable *and* per-statistic (most specific)
  - `'var'` — all statistics for a specific variable
  - `'stat'` — that statistic for all variables

- Lookup priority (applied in this order): (`var`,`stat`) → `var` → `stat`.

This logic ensures you can set global stat styles, per-variable styles, or very specific per-variable/stat styles — the most specific match wins.


In [7]:
# Custom format specifications for variables/statistics
format_specs = {
    # Per-variable formats (applies to all stats for that variable unless overridden)
    'wage': ',.1f',     # Wage always with 1 decimals
    # Per-variable/statistic formats (most specific, takes precedence)
    ('age', 'mean'): '.3f',   # Age mean with 3 decimals
    ('tenure', 'std'): '.4f', # Tenure std with 4 decimals
}

mt.DTable(
    df,
    vars=["wage", "age", "tenure"],
    stats=["mean", "std", "min", "max", "count"],
    format_spec=format_specs,
    caption="Custom formatting example with per-variable/statistic logic"
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8422900>

## Balance Tables with `BTable`
Balance Tables can be displayed with `BTable` which is based on `DTable` so inherits most of the latter's functionality. It constructs simple balance tables that shows variables by groups (like treatments in an experiment) and  performs statistical tests comparing these variables between the goups, displaying respective *p-values*. 

For two groups it displays the p-value of the single group indicator (t test) for more then two groups the p-value of a joint Wald test that all group indicators are zero is displayed. `BTable` uses `pyFixest` to perform the tests. You can add fixed_effects via `fixed_effects= ...` and specify the `vcov` option, for instance to implement clustering (see pyfixest documentation).    

In [8]:
mt.BTable(
    df,
    vars=["wage", "logwage", "age", "tenure"],
    group="worker_type",
    caption="Balance Table",
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1efc8423620>